## RunnableLambda, RunnableSequence and RunnableParallel

In [3]:
from langchain_cohere import ChatCohere
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence, RunnableLambda, RunnableParallel
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

In [4]:
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    
    cohere_api_key: str
    google_api_key: str

    class Config:
        env_file = "../.env"
        extra = "ignore"
        
settings = Settings()

In [5]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash-lite",
    google_api_key = settings.google_api_key,
    temperature = 0
)
llm

ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000231609DDD30>, default_metadata=(), model_kwargs={})

### 1. Multi-Step Sentiment Pipeline
#### Tags: RunnableSequence, RunnableLambda
#### Task: Create a sequence where:
1. Input is customer feedback text.
2. Step 1: Detect language (lambda).
3. Step 2: Translate to English if needed.
4. Step 3: Analyze sentiment.
5. Step 4: Output language + sentiment.

In [6]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are the customer feedback text analyzer."), 
        ("human", "{feedback}")
    ]
)
prompt_template

ChatPromptTemplate(input_variables=['feedback'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are the customer feedback text analyzer.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['feedback'], input_types={}, partial_variables={}, template='{feedback}'), additional_kwargs={})])

In [7]:
# prompt_chain = RunnableSequence(
#     prompt_template
#     | llm 
#     | StrOutputParser()
# )
# prompt_chain

In [8]:
from langdetect import detect

In [9]:
detect_language = RunnableLambda(
    lambda x: {
        "text": x["feedback"],
        "language": detect(x["feedback"])
    }
)

In [10]:
# detect_chain = RunnableSequence(
#     prompt_chain
#     | llm
#     | detect_language
# )
# detect_chain

In [11]:
language_translation = ChatPromptTemplate.from_messages(
    [
        ("system", "You are the feedback analyzer. If the human provides text in any language, translate it into English if not already in English."),
        # ("human", "{feedback}")
    ]
)
language_translation

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are the feedback analyzer. If the human provides text in any language, translate it into English if not already in English.'), additional_kwargs={})])

In [12]:
translate_chain = (
    language_translation 
    | llm
    | StrOutputParser()
)
translate_chain

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are the feedback analyzer. If the human provides text in any language, translate it into English if not already in English.'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000231609DDD30>, default_metadata=(), model_kwargs={})
| StrOutputParser()

In [13]:
analyze_sentiment = ChatPromptTemplate.from_messages(
    [
        ("system", "You are the sentiment analyzer. Read the following feedback and classify it as: **Positive**, **Negative**, and **Neutral**.."),
        ("human", "{text}")
    ]
)
analyze_sentiment

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are the sentiment analyzer. Read the following feedback and classify it as: **Positive**, **Negative**, and **Neutral**..'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [14]:
analyze_chain = (
    analyze_sentiment
    | llm
    | StrOutputParser()
)
analyze_chain

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are the sentiment analyzer. Read the following feedback and classify it as: **Positive**, **Negative**, and **Neutral**..'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000231609DDD30>, default_metadata=(), model_kwargs={})
| StrOutputParser()

In [15]:
chain = RunnableSequence(
    prompt_template
    | detect_language
    | RunnableLambda(lambda x: {"feedback": x["feedback"]}) 
    | translate_chain
    | RunnableLambda(lambda t: {"english_text": t})
    | analyze_chain

)

In [16]:
# combine_chain = RunnableSequence(
#     detect_language
#     | RunnableLambda(lambda x: {"feedback": x["feedback"], "language": x["language"], "text": x["feedback"]})
#     | translate_chain
#     | RunnableLambda(lambda x: {**x, "english_text": x["english_text"]})
#     | analyze_chain
# )


In [20]:
response = chain.invoke(
    {
        "feedback": "ഞാൻ ഒരു സ്മാർട്ട്ഫോൺ വാങ്ങി. അതിന്റെ ഗുണനിലവാരവും സവിശേഷതകളും നല്ലതാണ്."
    }
)
response

TypeError: 'ChatPromptValue' object is not subscriptable

### 2. Parallel News Analysis
#### Tags: RunnableParallel
#### Task: Given a news article, run three processes in parallel:
- Summarization
- Extract keywords
- Detect political bias

Return all three results in a dictionary.

In [21]:
summarization_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are the summarization assistant for news article to summerize the content. output is form a structure"),
        ("human", "summarize the following article {article}")
    ]
)

summarization_chain = summarization_prompt | llm | StrOutputParser()

In [22]:
extract_keyword_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are the expect of extract keyword assistant to the new article to extract the important keyword."),
        ("human", " Extract the keywords {article}")
    ]
)

extract_keyword_chain = extract_keyword_prompt | llm | StrOutputParser()

In [23]:
detect_political_bias_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are expert the media analyst. Read the following article and detect any political bias. Classify it as: Pro-Government, Anti-Government, Neutral, or Other."),
        ("human", "{article}")
    ]
)
detect_political_bias_chain = detect_political_bias_prompt | llm | StrOutputParser()

In [24]:
parallel_chain: dict = RunnableParallel(
    summarization = summarization_chain,
    extract_keyword= extract_keyword_chain,
    detect_political_bias = detect_political_bias_chain
)

In [25]:
response = parallel_chain.invoke(
    {
        "article": """
The government's new infrastructure development program has received widespread acclaim for boosting economic growth and creating thousands of jobs across the country. 
Experts say the plan will improve transportation, connectivity, and local businesses, strengthening the nation's economy. 
Opposition parties have raised minor concerns, but the majority of citizens have welcomed the initiative, praising the government for taking decisive steps toward modernization and national progress.
"""
    }
)
# print(response)
response

{'summarization': "**Summary of Infrastructure Development Program**\n\n*   **Key Benefit:** Boosts economic growth and creates thousands of jobs.\n*   **Expert Opinion:** Expected to improve transportation, connectivity, and local businesses, strengthening the national economy.\n*   **Public Reception:** Widely welcomed by the majority of citizens, who praise the government's decisive action towards modernization and national progress.\n*   **Opposition View:** Minor concerns have been raised by opposition parties.",
 'extract_keyword': "Here are the keywords extracted from the article:\n\n*   infrastructure development program\n*   economic growth\n*   job creation\n*   transportation\n*   connectivity\n*   local businesses\n*   nation's economy\n*   modernization\n*   national progress",
 'detect_political_bias': '**Classification:** Pro-Government\n\n**Analysis:**\n\nThe article presents a overwhelmingly positive view of the government\'s infrastructure development program. Here\'s

### 3. Job Application Analyzer
#### Tags: RunnableSequence, RunnableParallel
#### Task:
- Sequence: Parse job description -> extract skills.
- Parallel: Run “skill match score” and “recommend missing skills” at the same time.


In [26]:
job_description_template = ChatPromptTemplate.from_messages(
    [
        ("system", " You are the job description analyzer. To analyse a skills."),
        ("human", "{job_description}")
    ]
)

job_description_chain = job_description_template | llm | StrOutputParser()

In [27]:
skills_match_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a career assistant. Compare the candidate's skills with the job's required skills. Compute a skill match percentage (0-100%) indicating how well the candidate matches the job requirements."),
        ("human", "Candidate skill: {candidate_skills}")
    ]
)

In [28]:
skills_match_chain = skills_match_template | llm | StrOutputParser()

In [29]:
skills_missing_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a career advisor. Identify skills required by the job that the candidate does not have. Return a list of missing skills."),
        ("human", "Candidate skill: {candidate_skills}")
    ]
)


In [30]:
skills_missing_chain = skills_match_chain | llm | StrOutputParser()

In [31]:
final_chain = RunnableParallel(
    {
        "skill_score": skills_match_chain,
        "skill_missing": skills_missing_chain
    }
)
final_chain

{
  skill_score: ChatPromptTemplate(input_variables=['candidate_skills'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are a career assistant. Compare the candidate's skills with the job's required skills. Compute a skill match percentage (0-100%) indicating how well the candidate matches the job requirements."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['candidate_skills'], input_types={}, partial_variables={}, template='Candidate skill: {candidate_skills}'), additional_kwargs={})])
               | ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000231609DDD30>, default_metadata=(), model_kwargs={})
               | StrOutputParser

In [32]:
# response = job_description_chain.invoke(
#     {
#         "job_description": """
#     Position: Software Engineer  

# Responsibilities:
# - Develop and maintain web applications using Python and Django.
# - Collaborate with cross-functional teams to design scalable solutions.
# - Write clean, testable, and efficient code.

# Requirements:
# - Strong experience with Python and Django.
# - Familiarity with REST APIs, PostgreSQL, and Git.
# - Knowledge of Docker and Kubernetes is a plus.
# - Good problem-solving and communication skills.

#         """
#     }
# )
# print(response)

In [33]:
response = final_chain.invoke(
    {
        "job_description": """
    Position: Software Engineer  

Responsibilities:
- Develop and maintain web applications using Python and Django.
- Collaborate with cross-functional teams to design scalable solutions.
- Write clean, testable, and efficient code.

Requirements:
- Strong experience with Python and Django.
- Familiarity with REST APIs, PostgreSQL, and Git.
- Knowledge of Docker and Kubernetes is a plus.
- Good problem-solving and communication skills.

        """,
        "candidate_skills": """
        Name: Alex Johnson  

Skills:
- Python, Django, Flask
- REST APIs, MySQL
- Git, Linux
- Problem-solving, teamwork, communication
- Basic knowledge of Docker
"""
    }
)

In [34]:
print(response)

{'skill_score': 'Job required skill:\n        Job Title: Junior Python Developer\n\nSkills:\n- Python, Django, Flask\n- REST APIs, PostgreSQL\n- Git, Docker\n- Problem-solving, teamwork, communication\n- Experience with cloud platforms (AWS, Azure, GCP)\n- Unit testing', 'skill_missing': 'Here\'s a breakdown of the skills required for a Junior Python Developer, categorized for clarity and with explanations of why each is important:\n\n## Junior Python Developer - Required Skills\n\n**Core Programming & Frameworks:**\n\n*   **Python:**\n    *   **Why it\'s essential:** This is the foundational language. You need a solid understanding of Python\'s syntax, data structures (lists, dictionaries, tuples, sets), control flow (if/else, loops), functions, object-oriented programming (classes, inheritance), and common libraries.\n*   **Django:**\n    *   **Why it\'s important:** A high-level, "batteries-included" Python web framework. Experience with Django means you can build web applications e

### 4. Multi-Model Response Generator
#### Tags: RunnableParallel, RunnableLambda
#### Task:
Send the same user query to two different LLMs (e.g., GPT & Cohere) in parallel, then combine their responses into a single summarized answer using a lambda step.

In [35]:
llm1 = ChatCohere(cohere_api_key = settings.cohere_api_key)

In [45]:
multi_chatprompt1 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are the helpful assistant to user aked the question."),
        ("human", "{input}")
    ]
)

In [46]:
multi_chat_chain = multi_chatprompt1 | llm | StrOutputParser()

In [47]:
multi_chatprompt2 = ChatPromptTemplate.from_messages(
    [
        ("system", "you are the hlpfull assistant for user asked question"),
        ("human", "{input}")
    ]
)

In [48]:
multi_chat_chain2 = multi_chatprompt2 | llm1 | StrOutputParser()

In [50]:
parallel_chain: RunnableParallel = RunnableParallel(
    {
        "summary1" : multi_chat_chain,
        "summary2": multi_chat_chain2
    }
)

In [55]:
import json

In [63]:
response = parallel_chain.invoke("what is LLM in 2 points")
# print(json.dumps(response, indent=4))
print(response)

{'summary1': "Here's LLM in two points:\n\n1.  **LLM stands for Large Language Model.** It's a type of artificial intelligence designed to understand, generate, and process human language.\n2.  **They are trained on massive amounts of text data.** This extensive training allows them to learn patterns, grammar, facts, and reasoning abilities, enabling them to perform a wide range of language-related tasks.", 'summary2': "Here are 2 key points about **LLM (Large Language Model)**:\n\n1. **Definition**:  \n   An LLM is a type of artificial intelligence (AI) model designed to understand, generate, and manipulate human language. It is trained on vast amounts of text data to perform tasks like answering questions, writing text, translating languages, and more.\n\n2. **Key Features**:  \n   LLMs are characterized by their large size (billions of parameters), deep learning architecture (often based on transformers), and ability to generalize across diverse language tasks without task-specific 

### 5. Customer Support Ticket Classifier
#### Tags: RunnableSequence
#### Task:
1. Preprocess text (remove PII) via lambda.
2. Classify into department (billing, tech support, general).
3. Route to appropriate handler function.

### 6. Travel Planner with Dual Output
#### Tags: RunnableParallel, RunnableLambda
#### Task: Given a trip request, run in parallel:
- Generate a budget-friendly itinerary.
- Generate a luxury itinerary.
Then lambda step: Merge them into a “compare & choose” format.



6. Travel Planner with Dual Output
Tags: RunnableParallel, RunnableLambda
Task: Given a trip request, run in parallel:
- Generate a budget-friendly itinerary.
- Generate a luxury itinerary.
Then lambda step: Merge them into a “compare & choose” format.

7. FAQ Bot with Contextual Lookup
Tags: RunnableSequence, RunnableLambda
Task:
- Take user query -> embed & retrieve relevant docs from vector store -> pass to LLM for answer -> lambda step to add source citations.

8. Real-Time Social Media Monitor
Tags: RunnableParallel
Task: Given a batch of tweets:
- In parallel, run sentiment analysis, topic classification, and spam detection.
- Aggregate results into a dashboard-friendly JSON output.

9. Multi-Language Content Generator
Tags: RunnableSequence, RunnableParallel
Task:
- Sequence: Take a topic -> generate English content -> Parallel: translate into French, Spanish, Hindi at the same time.

10. AI Interview Assistant
Tags: RunnableSequence, RunnableParallel, RunnableLambda
Task:
- Sequence: Analyze candidate’s answer to a question -> evaluate soft skills score.
- Parallel: Check grammar, tone, and content relevance.
- Lambda: Combine all scores into a final report